In [1]:
# conda activate Ollama_RPG
# systemctl restart ollama


In [ ]:
%pip install ollama langchain-ollama

##### Bibliotecas e funções basicas

In [95]:
from langchain.output_parsers import PydanticOutputParser
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from pydantic import BaseModel
from typing import List
import random
import ollama # pega a lista de modelos que tem disponivel localmente no sistema
import json # se precisar manipular json
import html # para corrigir algumas saidas que vem com códigos de escape HTML, algo como &quot que representa " (aspas duplas).

In [96]:
models = ollama.list()
lista_modelos = []
i = 0
for model in models['models']:
    print(f"{i} - {model['model']}")
    lista_modelos.append(model['model'])
    i+=1

0 - phi3:3.8b-mini-128k-instruct-q8_0
1 - llama3.2:3b-instruct-q8_0
2 - qwen2.5:7b
3 - qwen2.5-coder:1.5b
4 - qwen2.5-coder:7b-instruct-q4_K_M
5 - nomic-embed-text:latest


In [97]:
modelo_id = 1
lista_modelos[modelo_id]

'llama3.2:3b-instruct-q8_0'

In [98]:
# Initialize Ollama
llm =  OllamaLLM(
    model=lista_modelos[modelo_id],
    keep_alive="30m"
)
def model_response(system_prompt, user_prompt, schema_json, output_class, modelo = llm):
    # Combine prompts with format instructions
    template = """{system_prompt}
        
    {user_prompt}

    {format_instructions}

    {schema_json}
    """
    # Create parser
    parser = PydanticOutputParser(pydantic_object=output_class)

    prompt = PromptTemplate(
        template=template,
        input_variables=["system_prompt", "user_prompt", "schema_json"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    # Generate formatted prompt
    formatted_prompt = prompt.format(
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        schema_json=schema_json
    )
    
    # Get response and parse
    response = modelo.invoke(formatted_prompt)
    try:
        parsed_response = parser.parse(response)
        return parsed_response
    except Exception as e:
        print(f"Error parsing response: {e}")
        return None

In [99]:
# Tabela de Tramas
Evento_inicial = [
    "Um exército de mortos avança",
    "Um tirano dominou terras",
    "Um mago controlou um grande mal",
    "Uma criatura poderosa despertou",
    "Um artefato sagrado foi roubado",
    "Um Rei bondoso foi assassinado",
    "Criaturas malignas estão se unindo",
    "Um local sagrado foi conspurcado",
    "Uma grande guerra se iniciou",
    "Um poderoso dragão surgiu",
    "Pessoas estão sendo assassinadas",
    "Um demônio invadiu nosso plano",
    "Está chovendo fogo dos céus",
    "Uma estranha torre surgiu do nada",
    "As divindades desapareceram",
    "As plantações estão morrendo",
    "Uma grande disputa iniciou",
    "Uma peste mágica se espalha",
    "Um velho louco contou algo",
    "Criaturas das trevas surgem a noite"
]

Acao_necessaria = [
    "Encontrar um item poderoso",
    "Despertar uma divindade",
    "Derrotar um Vilão",
    "Realizar uma grande viagem",
    "Invadir a poderosa fortaleza",
    "Resistir bravamente ao perigo",
    "Destruir algo perigoso",
    "Restabelecer a ordem",
    "Reunir um grande exército",
    "Resgatar o herói perdido",
    "Viajar para outro plano",
    "Desafiar a morte",
    "Caçar um monstro",
    "Encontrar algo perdido",
    "Descobrir um grande mistério",
    "Resistir ao medo e a loucura",
    "Buscar um componente raro",
    "Se aproximar furtivamente",
    "Estar sempre a frente",
    "Chegar em um local remoto"
]

Consequencia_terrivel = [
    "O mundo será destruído",
    "A escuridão tomará o mundo",
    "Um deus maligno despertará",
    "Suas terras serão tomadas",
    "Uma guerra eterna acontecerá",
    "Um reino de ódio começará",
    "Uma doença se espalhará",
    "Milhões morrerão",
    "O caos tomará o mundo",
    "A realidade irá colapsar",
    "Os deuses irão desaparecer",
    "Bestas dominarão o mundo",
    "Você perderá tudo o que tem",
    "A fome tomará o mundo",
    "Demônios tomarão o mundo",
    "Portais planares irão surgir",
    "Os mortos levantarão",
    "A magia desaparecerá",
    "Um mal passado retornará",
    "Uma noite eterna começará"
]

In [100]:
# Tabela de Cenas
Lugar = [
    "Uma pequena vila",
    "Um grande reino",
    "Uma masmorra mortal",
    "Um outro plano de existência",
    "Uma passagem perigosa",
    "Uma fortaleza inexpugnável",
    "Um local amaldiçoado",
    "Um local ermo",
    "Um local sagrado",
    "Uma movimentada taverna",
    "Uma viela escura",
    "Uma paisagem magnífica",
    "Um acampamento",
    "Uma feira movimentada",
    "Na estrada durante uma viagem",
    "Uma caverna inóspita",
    "No interior de um veículo",
    "Uma tribo selvagem",
    "Um labirinto",
    "No meio do nada"
]

Personagem_grupo_rival = [
    "Um aventureiro/grupo rival",
    "Uma pessoa traiçoeira",
    "Um velho aventureiro",
    "Um mensageiro",
    "Um ser muito poderoso",
    "Um exército",
    "Alguém amaldiçoado",
    "Um contratante misterioso",
    "Um artefato inteligente",
    "Uma pessoa muito rica",
    "Um animal",
    "Um conhecido do passado",
    "Uma tropa se deslocando",
    "Uma pessoa amiga/familiar",
    "Um batedor/espião",
    "Um fantasma ou desmorto",
    "Um viajante de terras distantes",
    "Um jovem escudeiro/ajudante",
    "Um familiar de um mago",
    "Uma criatura inteligente"
]

Evento = [
    "O Vilão surge",
    "Bandidos buscam riquezas",
    "Um grande mistério surge",
    "Alguém foi morto aqui",
    "Uma armadilha a frente",
    "Um ataque surpresa",
    "Os servos do Vilão estão aqui",
    "Uma passagem difícil/secreta",
    "Monstros errantes aparecem",
    "O Vilão ataca",
    "Você encontra pistas",
    "Você é cercado",
    "Uma armadilha a frente",
    "O Lugar está em perigo",
    "Monstros errantes aparecem",
    "O tempo muda drasticamente",
    "Os servos do Vilão estão aqui",
    "Algo o perturba",
    "Uma catástrofe acontece",
    "Você encontra um tesouro"
]

In [101]:
# Tabela de Raças
Racas = [# 12 Raças no total
    "Tiefling: Possui sangue infernal em suas veias (resistência ao fogo e visão no escuro)",
    "Gnomo: Perspicaz e astuto (resistência a ilusões e domínio de magias menores)",
    "Anão: Robusto e artesão nato (resistência a veneno e proficiência com ferramentas)",
    "Humano: Adaptável a toda situação e ambiente (receba Vantagem uma vez por Cena)",
    "Meio-Orc: Força e resistência sobre-humana (resistência a fadiga e ferimentos)",
    "Goliath: Forte como uma montanha (resistência ao frio e proficiência em alpinismo)",
    "Elfo: Graça e longevidade élfica (resistência a encantamentos e sentidos aguçados)",
    "Meio-Elfo: Versatilidade de dois mundos (escolha dois atributos para receber +1)",
    "Draconato: Herança dracônica (resistência elemental e proficiência com uma arma)",
    "Genasi: Afinidade elemental (resistência e proficiência relacionada ao elemento escolhido)",
    "Halfling: Pequeno e sortudo (furtividade e resistência a paralisia)",
    "Aasimar: Abençoado pelos celestiais (resistência radiante/necrótica e aura de cura)"
]

In [102]:
# Tabela de Classes
Classes = [ # 14 Classes
   "Ladino: Especialista em subterfúgio que vive em busca de riquezas, usando sua furtividade e astúcia",
   "Guerreiro: Mestre em combate versado em diversas armas e técnicas marciais",
   "Mago: Estudioso do oculto que controla poderosas magias através do conhecimento arcano",
   "Clérigo: Devoto divino que canaliza poder sagrado para espalhar a palavra de sua fé",
   "Monge: Lutador disciplinado que busca a perfeição através do equilíbrio entre corpo e mente",
   "Druida: Guardião da natureza que canaliza poderes selvagens para proteger as terras virgens",
   "Bárbaro: Feroz combatente que sobrevive nas terras selvagens através de sua força bruta",
   "Paladino: Soldado sagrado que luta incansavelmente por seus ideais e juramentos",
   "Artífice: Engenhoso inventor que combina magia e tecnologia para criar artefatos únicos",
   "Bruxo: Místico que obteve poderes através de um pacto com uma entidade extraplanar",
   "Patrulheiro: Batedor astuto especializado em rastrear e emboscar seus inimigos",
   "Bardo: Artista versátil que inspira aliados e eterniza as grandes histórias em canções",
   "Feiticeiro: Conjurador nato que manifesta magias poderosas através de seu próprio sangue",
   "Caçador de Sangue: Guerreiro sombrio que domina a arte proibida da magia sanguínea"
]

In [103]:
# Banco de ideias
Assunto = [
    "Um poderoso mago recluso",
    "Alguém importante está sumido",
    "Um herói do passado",
    "Um velho louco procura você",
    "Um distante templo sagrado",
    "Um exército se reúne",
    "Um povo de outrora",
    "Os bardos contam histórias",
    "Sinais de uma ruína perdida",
    "Uma guerra está acontecendo",
    "O caminho para um tesouro",
    "Uma doença se espalha",
    "Uma festa acontece/irá acontecer",
    "Uma fera deve ser caçada",
    "Uma mal cresce a cada dia",
    "Uma maldição foi lançada",
    "Os astros estão alinhados",
    "Uma grande mentira é contada",
    "Um tesouro está perdido",
    "Vultos estão cercando o local"
]

Acao = [
    "Viajar",
    "Unir/Reunir",
    "Prender",
    "Proteger",
    "Vencer/Derrotar",
    "Roubar/Resgatar",
    "Fugir",
    "Matar/Destruir",
    "Fortalecer",
    "Procurar",
    "Esconder",
    "Amar/Adorar",
    "Comer/Beber",
    "Construir",
    "Escalar/Subir",
    "Descansar/Parar",
    "Saltar/Descer",
    "Odiar/Temer",
    "Anular",
    "Teleportar"
]

Coisa_objeto = [
    "Tesouro",
    "Passagem secreta",
    "Cristal",
    "Sinal de violência",
    "Item mágico",
    "Rastros",
    "Portal",
    "Runa",
    "Ídolo/Estátua",
    "Animal mítico",
    "Roupas",
    "Gosma/Limo",
    "Fumaça/Névoa",
    "Garras/Escamas",
    "Garrafa/Poção",
    "Comida/Bebida",
    "Corpo/Destroço",
    "Dejetos",
    "Ferramentas",
    "Artefato"
]

Qualidade = [
    "Profano",
    "Frágil",
    "Bestial",
    "Real",
    "Divino",
    "Popular",
    "Perdido",
    "Mágico",
    "Resistente",
    "Sagrado",
    "Natural",
    "Pesado/Cheio",
    "Frio",
    "Invisível/Imaterial",
    "Nobre/Caro",
    "Energético",
    "Extraplanar",
    "Quente",
    "Leve/Vazio",
    "Ancestral"
]

In [104]:
# Para as cenas: Tem 4 lugares e e (eventos/personagens)
def cenas_aleatorias():
    """
    Seleciona as 4 cenas aleatórias para o RPG.
    sorteando 4 lugares e 4 eventos/personagens.
    """
    atos = ["Introdução", "Desenvolvimento", "Clímax Preparatório", "Desfecho"]
    L = random.sample(range(len(Lugar)), 4)
    lugar = [Lugar[L[0]], Lugar[L[1]], Lugar[L[2]], Lugar[L[3]]]
    
    personagens = [] # pode ser quan não apareça nenhum personagem.
    eventos = [] # pode ser quan não apareça nenhum evento.
    for i in range(4):
        dado = random.sample(range(1,21), 1)[0]
        if dado <= 10:
            P = random.sample(range(len(Personagem_grupo_rival)), 1)[0]
            personagens.append((atos[i], Personagem_grupo_rival[P]))
        else:
            E = random.sample(range(len(Evento)), 1)[0]
            eventos.append((atos[i], Evento[E]))

    return lugar, personagens, eventos

In [105]:
# Montar a Trama: seleciona a que aconteceu, o que precisa ser feito e as consequencias.


def cenario_aleatorio():
    AA = random.sample(range(len(Evento_inicial)), 1)[0]
    VP = random.sample(range(len(Acao_necessaria)), 1)[0]
    SN = random.sample(range(len(Consequencia_terrivel)), 1)[0]
    return f"""Descrisão do Cenário:
- Gênero: Fantasy
- Algo aconteceu: {Evento_inicial[AA]}.
- Você precisa: {Acao_necessaria[VP]}. 
- Consequências: {Consequencia_terrivel[SN]}.
- Ambientação: Você é um Aventureiro que se aventura em um incrível mundo de fantasia, rastejando em perigosas masmorras e enfrentando terríveis inimigos!"""

In [106]:
# Função que divide um texto em linhas menores, sem cortar palavras no meio.
import textwrap

def dividir_texto_em_linhas(texto, largura=60):
    """
    Divide um texto longo em linhas menores, sem cortar palavras no meio.

    Parâmetros:
        texto (str): O texto a ser dividido.
        largura (int): A largura máxima de cada linha. Padrão é 60 caracteres.

    Retorna:
        str: O texto formatado com quebras de linha.
    """
    # Divide o texto em linhas respeitando o limite de largura
    linhas = textwrap.wrap(texto, width=largura)
    
    # Junta as linhas com quebras de linha
    return "\n".join(linhas)

##### Ollama RPG

Criação da Introdução do cenário

In [121]:
cenario = cenario_aleatorio() # Seleciona o cenário e prepara o prompt
cenas = cenas_aleatorias() # Seleciona os locais e persoanges/eventos para as cenas

personagens = ""
for personagem in cenas[1]:
    personagens += f" {personagem[1]};"

locais = ""
for local in cenas[0]:
    locais += f" {local};"

system_prompt = """Você é um especialista em criação de histórias para RPG."""

promtpt_introducao = f"""Crie a introdução de uma aventura de RPG em um mundo de fantasia que ajude na construção das cenas. Com base na Descrição a seguir:

## Descrição:
{cenario}

## Instruções:
1. Descrição da ambientação do mundo, baseado na Descrição do cenário **Ambientação** e **Gênero**.
2. Descrição de como o mundo está agora, baseado na Descrição do cenário **Algo aconteceu**.
3. Descrição da ação necessária para resolver o estado atual, baseado na Descrição do cenário **Você precisa**.
4. Descrição das consequências caso o objetivo não seja cumprido, baseado na Descrição do cenário **Consequências**.
5. Não defina personagens nem eventos.
6. Defina um nome para os locais de acordo com a sua descrição, para os seguintes as seguintes descrisções: {locais}
7. Conecte as partes da historia de maneira logica e coerente.

## Saida
Você deve responder em formato JSON seguindo o seguinte esquema:\n"""


schema_json_string = "{\n"
schema_json_string += "  \"messages\": [\n"
schema_json_string += "    {\"number\": 1, \"content\": \"Descrição breve da ambientação do mundo (nome dos lugares, clima, cultura, povos) sem mensionar o que ocorrerá no futuro ou quem fará a mudança. Sempre iniciando com: 'Em um mundo onde ...'.\"},\n"
schema_json_string += "    {\"number\": 2, \"content\": \"Descrição breve como o mundo está **agora**, destacando o que mudou, e explicar em detalhes o evento ou personagem que levou o mundo a esse estado. Sempre iniciando com: 'Mas agora, ...'\"},\n"
schema_json_string += "    {\"number\": 3, \"content\": \"Descrição breve da ação necessária para resolver o estado atual, com no máximo 1 ação ou objetivo. Não inclua consequências do fracasso nem quem deve executar a ação. Sempre comece com: 'É necessário ...'\"},\n"
schema_json_string += "    {\"number\": 4, \"content\": \"Descrição breve das consequências caso o objetivo não seja cumprido, ligando ela **ao que aconteceu**.\"}],\n"
schema_json_string += "  \"antagonist\": \"Descrição do objetivo do antagonista, começando com: 'O objetivo do antagonista é...'\",\n"
schema_json_string += "  \"locations\": [\n"

for i in range(len(cenas[0])): 
    schema_json_string += f"    {{\"description\": \"{cenas[0][i]}\", \"name\": \"Nome para um local descrito como: '{cenas[0][i]}'\"}}"
    if i < len(cenas[0]) - 1: 
        schema_json_string += ","
    schema_json_string += "\n"

schema_json_string += "  ],\n"
schema_json_string += "  \"objective\": \"Informe brevemente o objetivo principal, começando com: 'O objetivo principal é ...'\"\n"
schema_json_string += "}"


introducao_schema = json.loads(schema_json_string)

class Message(BaseModel):
    number: int
    content: str

class Location(BaseModel):
    description: str
    name: str
    
# Classe que representa o cenário completo
class Story(BaseModel):
    messages: List[Message]
    antagonist: str
    locations: List[Location]
    objective: str

story = model_response(system_prompt, promtpt_introducao, introducao_schema, Story)

print(cenario)
print("\nIntrodução:")
for message in story.messages:
    print(dividir_texto_em_linhas(message.content, 100))
    print()
print(f"Objetivo principal: {story.objective}")
print(f"\nAntagonista: {story.antagonist}")
print(f"\nLocais:")
for location in story.locations:
    print(f">> {location.name}: {location.description}")

Descrisão do Cenário:
- Gênero: Fantasy
- Algo aconteceu: Está chovendo fogo dos céus.
- Você precisa: Caçar um monstro. 
- Consequências: Um reino de ódio começará.
- Ambientação: Você é um Aventureiro que se aventura em um incrível mundo de fantasia, rastejando em perigosas masmorras e enfrentando terríveis inimigos!

Introdução:
Em um mundo onde a magia é uma parte integral da vida, os reinos de Aerthys e Valtoria coexistem em
harmonia. O clima é temperado, com verões quentes e invernos frios, e as culturas são ricas e
variadas. Os povos vivem em cidade-estados unidas sob a coroa de um monarca justo.

Mas agora, o céu está queimando com fogo constante, e os reinos estão se dividindo por ódio e medo.
A magia que uma vez unificou a região agora é vista como uma ameaça, e os povos estão sendo forçados
a escolher entre a morte ou a subjugação.

É necessário caçar o monstro conhecido como 'O Fogo do Céu', que é a causa imediata do mal. O local
onde o monstro reside é desconhecido, mas se

Detalhes sobre os persenagens.

In [169]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

personagens_descrisao = ""
for i in range(len(cenas[1])):
   personagens_descrisao += f"{cenas[1][i][1]}; "
personagens_descrisao += f"O Antagonista, {story.antagonist}."
print(personagens_descrisao)

promtpt = f"""Para todos os personagens, escolha um nome, papel, motivation, personality e Connection to Tram de acordo com a sua importância na história.

## Introdução:
1. **Ambientação**:   
{story.messages[0].content}

2. **O que aconteceu**: 
{story.messages[1].content}

3. **Como resolver**: 
{story.messages[2].content}

4. **Consequências de Falhar**:  
{story.messages[3].content}

5. **Objetivo Principal**:
{story.objective}

6. **Tabela de Elementos**:

| **Prefixo**        | **Raiz**         | **Sufixo**      |
|---------------------|------------------|-----------------|
| Aure (Dourado)     | Magnus (Grande)  | ius (Homem)     |
| Luci (Luz)         | Noct (Noite)     | us (Ser)        |
| Cor (Coração)      | Fulg (Brilho)    | arius (Guardião)|
| Flavi (Amarelo)    | Tempest (Tempestade) | or (Aquele que) |
| Cael (Céu)         | Ignis (Fogo)     | ix (Portador)   |
| Domin (Senhor)     | Tenebris (Trevas)| ens (Essência)  |
| Viri (Força)       | Mortem (Morte)   | os (Ligado a)   |
| Pax (Paz)          | Bellum (Guerra)  | arum (Herdeiro) |
| Aqua (Água)        | Vita (Vida)      | ana (Sagrado)   |
| Terra (Terra)      | Lumina (Luz)     | ion (Criador)   |


## Istruções para formar nomes:
1. Estrutura de Nomes: Um nome é formado pela combinação de até três elementos:
    - Prefixo: Representa uma ideia principal, como uma característica ou essência (ex.: Aure - Dourado).
    - Raiz: Representa o núcleo semântico do nome, frequentemente relacionado a um conceito central (ex.: Ignis - Fogo).
    - Sufixo: Indica um atributo ou papel, muitas vezes definindo a função ou estado do personagem (ex.: arius - Guardião).

2. Formas de Combinação: O modelo deve combinar os elementos seguindo estas estruturas ou variantes:
    - Prefixo + Raiz + Sufixo. Exemplo: Auremagnusius (Homem Dourado e Grande).
    - Raiz + Sufixo + Prefixo. Exemplo: IgnisariusCael (Guardião do Fogo do Céu).
    - Prefixo + Sufixo + Raiz. Exemplo: DominensTenebris (Essência do Senhor das Trevas).
    - Raiz + Prefixo + Sufixo. Exemplo: MortemVirius (Homem de Força e Morte).

3. Regras para a Criação de Nomes: 
    - Os elementos podem ser combinados livremente em qualquer ordem para criar nomes estilizados, desde que mantenham um significado coerente.
    - É permitido modificar ligeiramente as terminações para melhorar a fluidez do nome (ex.: mudar "us" para "um").
    - O significado resultante do nome deve ser inspirado pelas palavras originais e refletir um conceito que seja adequado a personagens de RPG, como guerreiros, magos ou entidades sobrenaturais.

4. Atribua a cada um do personagens apenas um dos seguintes papeis: {personagens_descrisao} 



## Saída Esperada:
"""

personagens_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"characters\": [\n"
"    {\n"
"      \"name\": \"nome para o personagem com base nas **Instruções para formar nomes**.\",\n"
"      \"role\": \" Um dos papeis informados.\",\n"
"      \"motivation\": \"Detalhe o que move o personagem em relação à história principal, como vingança, ambição, dever, ou sobrevivência.\",\n"
"      \"personality\": \"Descreva os traços principais de comportamento e atitudes do personagem.\",\n"
"      \"connection_tram\": \"Especifique como o personagem está ligado aos eventos ou objetivos principais da aventura.\",\n"
"    }\n"
"  ]\n"
"}"
)

class Character(BaseModel):
    name: str
    role: str
    motivation: str
    personality: str
    connection_tram: str

class Characters(BaseModel):
    characters: List[Character]

personagens_cenas = model_response(system_prompt, promtpt, personagens_schema, Characters)

personagens_cenas.model_dump()

Um mensageiro; Uma tropa se deslocando; O Antagonista, O objetivo do antagonista é extinguir a magia e unificar a região sob sua vontade, usando o medo e a violência para alcançar seu objetivo..


{'characters': [{'name': 'Auremagnusiusscori',
   'role': 'Um mensageiro',
   'motivation': 'Vingança pessoal contra aqueles que se opõem à magia, pois acredita que a magia é uma força divina e necessária para o bem-estar da região.',
   'personality': 'Auremagnusiusscori é um mensageiro leal, determinado e corajoso. É conhecido por sua habilidade em navegar pelas ruas mais perigosas e por sua capacidade de manter a calma em situações de crise.',
   'connection_tram': 'Ele está ligado ao objetivo principal da aventura como mensageiro do reino de Aerthys, que precisa comunicar informações sobre o local onde se encontra o Fogo do Céu.'},
  {'name': 'IgnisariusCaeli',
   'role': 'Um guerreiro que luta contra o Fogo do Céu',
   'motivation': 'Acredita que é sua missão proteger os innocentês da região e combater aqueles que buscam destruir a magia.',
   'personality': 'IgnisariusCaeli é um guerreiro destemido e corajoso, com habilidades mágicas excepcionais. É conhecido por sua força e capa

In [194]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

lugares = ""
atos = ["Introdução", "Desenvolvimento", "Clímax Preparatório", "Desfecho"]
i = 0 # indice para a cena
j=0
for local in story.locations:
    lugares += f"* {atos[i]}: {local.name} ({cenas[0][i]}) "
    
    # Procura por personagem no local i
    for even, personagem in cenas[1]:
        if atos[i] == even:
            if "ntagonista" not in personagens_cenas.characters[j].role:
                lugares += f"""onde você encontra {personagens_cenas.characters[j].name}, {personagens_cenas.characters[j].role}: {personagens_cenas.characters[j].connection_tram}"""
                j += 1
            
    # Procura por evento no local i
    for even, evento in cenas[2]:
        if even == atos[i]:
            if i ==3:
                lugares += f"onde {evento} e onde você encontra {personagens_cenas.characters[j].name}, {personagens_cenas.characters[j].role}: {personagens_cenas.characters[j].connection_tram}"
            else:
                lugares += f"onde {evento}"
    lugares += "\n"
    i += 1

nome_locais = ""
for local in story.locations:
   nome_locais += f"{local.name}; "

promtpt = f"""Para cada um dos seguintes locais: {nome_locais}, defina as seguintes informações detalhadas:

## Introdução:
1. **Ambientação**:   
{story.messages[0].content}

2. **O que aconteceu**: 
{story.messages[1].content}

3. **Como resolver**: 
{story.messages[2].content}

4. **Consequências de Falhar**:  
{story.messages[3].content}

5. **Lugares**:
{lugares}

## Instruções:
1. Os locais {nome_locais} devem estar presentes obrigatoriamente.
2. **Descrição do Local** inclua elementos visuais (o que se vê), auditivos (sons predominantes) e atmosféricos (sensação ou emoção gerada pelo ambiente).
3. **Papel do Local na Aventura**, explique o que ira acontecer neste local e como esse acontecimento contribui para a progressão da história de acordo com a cena em que está localizado:
    - Introdução: Nesta cena apresenta o contexto inicial, o conflito ou informações essenciais para começar a aventura.
    - Desenvolvimento: Nesta cena ajuda a avançar a trama inicial, como lugares que abrigam aliados, inimigos ou desafios importantes.
    - Clímax Preparatório: Nesta cena aumenta a tensão ou preparam os heróis para o confronto final.
    - Desfecho: Nesta cena tem o confronto final ou da resolução da aventura. Certifique-se de que pelo menos um dos locais seja designado para esta cena.

## Saída Esperada:
"""

locations_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"locations\": [\n"
"    {\n"
"      \"name\": \"Informe apenas o nome marcante para o local.\",\n"
"      \"role\": \"Uma descrição detalhada do principal acontecimento no local.\",\n"
"      \"description\": \"Descrição detalhada, com elementos visuais, auditivos e atmosféricos do local\",\n"
"    }\n"
"  ]\n"
"}"
)

class Scene(BaseModel):
    name: str
    purpose: str

class Location(BaseModel):
    name: str
    role: str
    description: str

class Locations(BaseModel):
    locations: List[Location]

locais_cenas = model_response(system_prompt, promtpt, locations_schema, Locations)
print(f"Total de lugares: {len(locais_cenas.locations)} - {nome_locais}\n")

for local in locais_cenas.locations:
    print(f"{local.name}")
    print(f"    - Papel: {local.role}")
    print(f"    - Descrição: {local.description}")

Total de lugares: 4 - Cidade da Escuridão; Temple de Valtoria; Labirinto de Aerthys; Estrada da Perda; 

Cidade da Escuridão
    - Papel: Introdução à aventura, onde o mensageiro Auremagnusiusscori fornece informações sobre o Fogo do Céu e o objetivo principal da missão.
    - Descrição: Uma cidade amaldiçoada com ruas estreitas e sombrias, onde a atmosfera é pesada e as sombras parecem se mover por conta própria. O cheiro de fumaça e carvão pode ser sentido no ar, e as pessoas andam com cautela, como se temesse o olhar do destino. Auremagnusiusscori espera no portão da cidade, vestido com roupas escuras e olhos que parecem ver além das sombras.
Temple de Valtoria
    - Papel: Encontro com o guerreiro IgnisariusCaeli, um aliado na luta contra o Fogo do Céu.
    - Descrição: Um templo grandioso e impressionante, com paredes cobertas de símbolos antigos e frescas pinturas que contam a história da batalha contra o Fogo do Céu. O som dos soares e das orações pode ser ouvido no interior do 

In [214]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

locais_das_cenas = ""
atos = ["Introdução", "Desenvolvimento", "Clímax Preparatório", "Desfecho"]
i=0
for locais in locais_cenas.locations:
    locais_das_cenas+= f"* {atos[i]}: {locais.name}, {locais.role} É {locais.description}\n"
    i+=1
    
lista_pessoas = ""
i = 0 # indice para a cena
for character in personagens_cenas.characters:
    lista_pessoas += f"* {atos[i]}: {character.name} - {character.role}: {character.connection_tram}" 
    lista_pessoas += "\n"
    i += 1
promtpt = f"""Divida o objetivo principal da aventura: {story.objective}.

## Introdução: # 
1. **O que aconteceu**: 
{story.messages[1].content}

2. **Como resolver**: 
{story.messages[2].content}

3. **Objetivo Principal**:
{story.objective}

4. **Locais Importantes**:
{locais_das_cenas}

5. **Personagens Importantes**:
{lista_pessoas}

## Instruções:
1. Divida o objetivo principal em 4 cenas: Introdução, Desenvolvimento, Clímax Preparatório e Desfecho.
2. Para cada cena, defina:
    - Objetivo da Cena: O que os jogadores devem alcançar ou fazer.
    - Como realizar o objetivo: Como os jogadores podem cumprir a tarefa ou desafio da cena.
    - Obstáculos: Quais desafios ou dificuldades eles enfrentarão.
    - Conexão com a Próxima Cena: Como a cena leva à próxima.
3. Mantenha a progressão da narrativa lógica, garantindo que cada cena avance para a resolução final.
4. Seja direto e claro, explicando a conexão entre as cenas.

## Saída Esperada:"""

objective_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"objectives\": [\n"
"    {\"scenes\":  \"Introdução\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"    {\"scenes\":  \"Desenvolvimento\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"    {\"scenes\":  \"Clímax Preparatório\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"    {\"scenes\":  \"Desfecho\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"  ]\n"
"}"
)

class objective(BaseModel):
    scenes: str
    objective: str
    description: str
    Connection_next_scene: str
    
class Objectives(BaseModel):
    objectives: List[objective]

res = model_response(system_prompt, promtpt, objective_schema, Objectives)
res.model_dump()

{'objectives': [{'scenes': 'Introdução',
   'objective': 'O objetivo dessa cena é informar brevemente o início do jornada, explicando que o objetivo principal é caçar o monstro e parar a catástrofe que ameaça consumir a região.',
   'description': 'A cena começa com Auremagnusiusscori, um mensageiro do reino de Aerthys, que fornece informações sobre o Fogo do Céu e o objetivo principal da missão.',
   'Connection_next_scene': 'Após a introdução, os personagens são apresentados à cidade da escuridão, onde estão preparados para enfrentar as desafios que se encontram.'},
  {'scenes': 'Desenvolvimento',
   'objective': 'O objetivo dessa cena é apresentar os heróis ao Temple de Magia, onde eles aprenderão mais sobre a magia e seus poderes.',
   'description': 'Os personagens visitam o Temple de Magia, onde são recebidos por um sábio que lhes ensina sobre a importância da magia na preservação da região.',
   'Connection_next_scene': 'Depois do aprendizado nos ensinamentos do sábio, os heróis

In [809]:
for objetivo in res.objectives:
    print(objetivo.objective)

Encontrar informações básicas sobre a origem do problema das criaturas das trevas e identificar pistas essenciais para avançar na missão
Recuperar um item essencial que lhe permitirá superar obstáculos e alcançar o objetivo principal da missão
Negociar com uma figura importante que lhe fornecerá informações críticas sobre a missão e seus objetivos
Confrontar o Antagonista, derrotá-lo e salvar o mundo das criaturas das trevas


In [797]:
print(story.objective)
print()
for objetivo in res.objectives:
    print(objetivo.Connection_next_scene)

Encontrar o Componente Raro para derrotar Xaren, o Bruxo das Trevas e salvar o mundo das criaturas das trevas.

A conexão dessa cena com a próxima é que os jogadores precisam levar as informações coletadas até o Castelo de Arete, onde encontrarão um aliado importante.
A conexão dessa cena com a próxima é que os jogadores precisam levar o item recuperado para o Submundo das Sombras.
A conexão dessa cena com a próxima é que os jogadores precisam levar os recursos obtidos para a Fortaleza das Trevas.



Divisão dos arcos da aventura

In [168]:
# Prepara os pontos basicos dos 4 arcos
lugares = ""
for local in introducao.locations:
   lugares += f"* {local.name} ({local.purpose}): {local.evento}\n"

personagens = ""
for personagem in introducao.personagens:
   personagens += f"* {personagem.name}: {personagem.role}\n"


prompt_arco = f"""Sua tarefa é ajudar a criar narrativas divididas em 4 cenas claros e progressivos, utilizando os dados fornecidos em **Introdução (Dividida em Partes)** e seguindo as diretrizes a seguir:


## Introdução (Dividida em Partes): 
1. **Ambientação**:   
{introducao.messages[0].content}

2. **O que aconteceu**: 
{introducao.messages[1].content}

3. **Como resolver**: 
{introducao.messages[2].content}

4. **Consequências de Falhar**:  
{introducao.messages[3].content}

5. **Objetivo principal**:  
{introducao.objective}

## Locais Importantes:
{lugares}  

## Personagens Importantes:
{personagens}  

## Instruções:  
1. **Seleção de Locais**: Para cada cena selecione o loclal onde ela ira ocorrer. 
   - Priorize os **Locais Importantes** .  
   - Varie os tipos de locais entre os cenas para criar uma progressão interessante.  

2. **Seleção de Personagens**: Para cada cena selecione um ou mais personagem que irá participar.  
   - Priorize os **Personagens Importantes**. Caso não haja personagens definidos, crie personagens que façam sentido na história. 

3. **Seleção de Eventos**: Para cada cena selecione um evento que ocorra nesse local. 
   - Priorize os evento sugeridos para os locais. Caso não haja escolha eventos que promovam tensão narrativa e reflitam a progressão do arco.  
   - Certifique-se de que os eventos conectem-se diretamente ao objetivo principal.  
   - Relacione os eventos às partes da introdução, conforme indicado abaixo.  

4. **Definição de Objetivos**: Para cada cena defina um objetivo específico. Que seja parte do **objetivo principal**.  
   - Divida o objetivo principal em etapas claras e progressivas ao longo dos 4 cenas.  
   - Cada objetivo deve levar à próxima fase da história, culminando na resolução da trama no arco final.  
   - Relacione os objetivos ao local, personagem e evento do arco.  

5. **Conexão Geral**: Conecte as escolhas para criar um narrativa coesa. 
   - Certifique-se de que as escolhas de locais, personagens, eventos e objetivos conectem-se logicamente e criem uma narrativa coesa. 
   - Certifique-se de que cada arco introduza elementos ou personagens que contribuam diretamente para os eventos ou resoluções nos cenas seguintes, criando uma progressão temática clara e coesa.
   - Escale a tensão e a complexidade dos cenas à medida que a história avança, culminando em um clímax na Cena 4.  

## Saída Esperada:  
"""

arcos_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"cenas\": [\n"
"    {\n"
"      \"name\": \"Cena 1: Introdução\",\n"
"      \"location\": \"Defina o local onde a cena inicial ocorre, refletindo a ambientação descrita na introdução.\",\n"
"      \"character\": \"Liste os personagens principais apresentados nesse momento, geralmente aqueles que introduzem o conflito ou contexto.\",\n"
"      \"event\": \"Descreva o cenário inicial e o incidente que dá início à aventura, alinhando-se com a 'Ambientação' e 'O que aconteceu' na introdução.\",\n"
"      \"objective\": \"Apresente o primeiro desafio ou mistério que os jogadores precisam compreender para seguir na história.\",\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 2: Obstáculo Inicial\",\n"
"      \"location\": \"Defina o local onde ocorre o primeiro grande desafio, conectado ao objetivo de encontrar o artefato ou avançar na missão.\",\n"
"      \"character\": \"Inclua aliados ou inimigos que os jogadores encontram, com foco em desenvolver a narrativa inicial.\",\n"
"      \"event\": \"Explique o primeiro grande obstáculo e como ele reflete os passos iniciais para resolver o problema principal, baseado em 'Como resolver'.\",\n"
"      \"objective\": \"Estabeleça o que os jogadores precisam superar nesse arco para seguir na história, como encontrar informações ou superar uma ameaça inicial.\"\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 3: Escalada\",\n"
"      \"location\": \"Descreva o local onde os eventos se tornam mais complexos ou perigosos, refletindo a progressão da história.\",\n"
"      \"character\": \"Apresente personagens que aumentam a tensão da trama, como inimigos mais poderosos ou aliados em risco.\",\n"
"      \"event\": \"Detalhe como o conflito se intensifica, ligando aos elementos de 'Como resolver' e 'Consequências de Falhar'.\",\n"
"      \"objective\": \"Defina o que os jogadores precisam realizar para evitar grandes perdas e preparar-se para o clímax da aventura.\",\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 4: Clímax e Resolução\",\n"
"      \"location\": \"Descreva o local onde acontece o confronto final ou a resolução da história, conectando ao objetivo principal.\",\n"
"      \"character\": \"Inclua o antagonista principal ou aliados cruciais que desempenham papéis essenciais no desfecho.\",\n"
"      \"event\": \"Explique o confronto ou evento final, com foco nas 'Consequências de Falhar' e no 'Objetivo principal'.\",\n"
"      \"objective\": \"Estabeleça o que os jogadores precisam alcançar para concluir a aventura com sucesso, como derrotar o vilão ou proteger o artefato.\",\n"
"    }\n"
"  ]\n"
"}"
)

class Cena(BaseModel):
   name: str
   location: str
   character: str
   event: str
   objective: str

class Cenas(BaseModel):
   cenas: List[Cena]


system_prompt = """Você é um especialista em criação de histórias para RPG. Siga as instruções fornecidas e responda em formato JSON seguindo o esquema especificado."""

In [169]:
base_arcos = model_response(system_prompt, prompt_arco, arcos_schema, Cenas)
base_arcos.model_dump()

{'cenas': [{'name': 'Cena 1: Introdução',
   'location': 'Defina o local onde a cena inicial ocorre, refletindo a ambientação descrita na introdução.',
   'character': 'Liste os personagens principais apresentados nesse momento, geralmente aqueles que introduzem o conflito ou contexto.',
   'event': "Descreva o cenário inicial e o incidente que dá início à aventura, alinhando-se com a 'Ambientação' e 'O que aconteceu' na introdução.",
   'objective': 'Apresente o primeiro desafio ou mistério que os jogadores precisam compreender para seguir na história.'},
  {'name': 'Cena 2: Obstáculo Inicial',
   'location': 'Defina o local onde ocorre o primeiro grande desafio, conectado ao objetivo de encontrar o artefato ou avançar na missão.',
   'character': 'Inclua aliados ou inimigos que os jogadores encontram, com foco em desenvolver a narrativa inicial.',
   'event': "Explique o primeiro grande obstáculo e como ele reflete os passos iniciais para resolver o problema principal, baseado em 'Co

In [170]:
print(prompt_arco)

Sua tarefa é ajudar a criar narrativas divididas em 4 cenas claros e progressivos, utilizando os dados fornecidos em **Introdução (Dividida em Partes)** e seguindo as diretrizes a seguir:


## Introdução (Dividida em Partes): 
1. **Ambientação**:   
Em um mundo onde a natureza parece ter perdido seu equilíbrio, os reinos de Eldrador e Valtoria se estendem como faróis no céu noturno. As montanhas majestosas de Eldrador oferecem abrigo às tribos nômades que vivem em harmonia com a terra, enquanto as florestas mágicas de Valtoria escondem segredos antigos. As cidades-estado de Calonia e Arkeia brilham como joias no deserto, suas torres de cristal refletindo a luz do sol ao pôr-do-sol. A cultura é rica e diversificada, com uma mistura única de tradições e influências estrangeiras.

2. **O que aconteceu**: 
Mas agora, o mundo está mudo. Um exército de mortos avança em direção às cidades-estado, seu barulho ecoa pelas montanhas e florestas como um rugido de fera. A magia que mantinha os mort

Criação das cenas

In [128]:
## Cena 1: Introdução e Contexto
introducao_completa = texto_introducao
local_evento_atual = "Uma pequena vila e envolve Um grande mistério"
proximo_local_evento = "Um grande reino e envolve O Vilão ataca"
# {local} e envolve {evento}

prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Tipo de Cena
Esta cena apresenta o cenário, o contexto e o gancho principal da história. Seu objetivo é colocar o personagem em movimento, mostrando a crise que ele deve resolver e respondendo a perguntas como: O que está em jogo? Quem ou o que causa o problema? Qual é o objetivo inicial do personagem?  

### Instruções 
1. Priorize **locais, eventos e personagens** mencionados na introdução.  
2. Caso a introdução não forneça informações suficientes, utilize as sugestões abaixo como fallback:  
   - **Local  e Evento Atual**: {local_evento_atual}  
   - **Próximo Local e Evento**: {proximo_local_evento} 
3. **Twists e Obstáculos**: Sugira até dois twists ou obstáculos inesperados para aumentar a tensão e manter o interesse.
4. Seja consistente e coerente ao definir o objetivo da cena atual e o objetivo para o proximo local.
5. **Extração de Informações** extraia exclusivaemente do Contexto da Aventura os seguintes dados:
  - **Itens importantes**: Informe o nome, função e condição de uso para os itens importantes para a trama principal e o seu status: "Não encontrado", "Não utilizado" e "Usado".
  - **Personagens importantes**: Informe o nome, papel e motivações para os personagens para a trama principal e o se sua participação ja foi concluida: "Não apareceu", "Não concluido" e "Concluido".
  - **Assuntos importantes**: Informe o nome, a da descrisão dos assuntos importantes para a trama principal e status se ele ja foi ou não tratado: "Não resonvido", "resonvido".

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""
narrativa_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"name_locale\": \"Nome do local\",\n"
    "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\",\n"
    "  \"objetivo_description\": \"Explique o objetivo do personagem com uma ação iniciada no verbo no infinitivo\",\n"
    "  \"Connection_next_location\": \"Indique o próximo local no local atual com uma ação no imperativo e introduza um objetivo para o próximo local\",\n"
    "  \"itens\": [\n"
    "    {\"name\": \"Nome do item\", \"function\": \"Função do item\", \"condition_use\": \"Condição de uso do item\", \"status\": \"Status do item\"}\n"
    "  ],\n"
    "  \"personagens\": [\n"
    "    {\"name\": \"Nome do personagem\", \"role\": \"Papel do personagem\", \"motivation\": \"Motivação/segredos do personagem\", \"participation\": \"Seu participação ja foi concluida\"}\n"
    "  ],\n"
    "  \"assuntos\": [\n"
    "    {\"name\": \"Nome para o assunto\", \"description\": \"Breve explicação sobre o assunto\", \"status\": \"Status do assunto\"}\n"
    "  ],\n"
    "  \"twists\": [\n"
    "    {\"description\": \"Descrição da reviravolta\"}\n"
    "  ]\n"
    "}"
)

In [107]:
# Cena 2: Continuação da Cena Anterior
introducao_completa = texto_introducao
objetivo_anterior = "Investigar as mortes e desaparecimentos" # Objetivo da cena anterior (foi concuido)
local_evento_atual = "Partir em direção às Montanhas de Ardesia em busca dos Símbolos da Divindade" #(novo objetivo)


prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Próxima Etapa da Jornada
Após {objetivo_anterior}, a história avança para {local_evento_atual}, onde um novo desafio ou descoberta impede o personagem de alcançar seu objetivo diretamente.

### Instruções
1. Respeite os **objetivos definidos**:
   - O **objetivo atual** deve ser cumprido **neste local** e não deve ser alterado ou substituído.
   - A **conexão com o próximo local** deve apresentar uma **ação nova** que avança a trama, sem repetir o objetivo atual.
   - A conexão com o próximo local deve ser independente e não pode interferir ou impedir a conclusão do objetivo atual.
2. **Local e Evento**:
   - O local deve ser descrito com **detalhes sensoriais** (visuais, sons, cheiros) para criar uma atmosfera imersiva.
   - O evento principal deve introduzir personagens, obstáculos ou conflitos que desafiam o personagem a cumprir seu objetivo.
3. **Twists e Obstáculos**:
   - Sugira até dois twists ou obstáculos inesperados que dificultem ou compliquem a jornada do personagem.

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""

narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Apresente uma ação nova para o próximo local, evitando repetir o objetivo atual. A conexão deve avançar a trama com um novo desafio ou descoberta.\"\n,"
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [110]:
# Cena 3: Continuação da Cena Anterior para o clímax
introducao_completa = texto_introducao
objetivo_anterior = "Investigar o local e encontrar pistas sobre os Símbolos da Divindade" # Objetivo da cena anterior (foi concuido)
local_evento_atual = "Solicitar informações à figura encapuzada sobre os símbolos" #(novo objetivo)


prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Próxima Etapa da Jornada
- Após cumprir o objetivo anterior: **{objetivo_anterior}**, o personagem agora deve avançar para: **{local_evento_atual}**.
- Esta cena deve preparar a transição direta para o **clímax da trama principal**, que será resolvido na próxima etapa.

### Instruções
1. **Objetivo da Cena**:
   - O personagem deve cumprir o **objetivo definido** nesta cena, sem modificá-lo ou desviá-lo: {local_evento_atual}.
   - A ação nesta cena deve ser **essencial** para avançar a história em direção ao clímax.

2. **Descrição do Local e Evento**:
   - Descreva o local com **detalhes sensoriais ricos** (visuais, sons, cheiros) para criar uma atmosfera imersiva.
   - Introduza um **evento principal** com personagens ou obstáculos que desafiem o personagem a cumprir o objetivo.

3. **Conexão com o Clímax**:
   - Apresente o **próximo local** com uma ação clara que leve diretamente ao **clímax da aventura**.
   - A conexão deve ser lógica e preparar a resolução da trama principal.
   - A conexão com a proxima cena deve ser diferente do objetivo anterior e objetivo atual

4. **Twists e Obstáculos**:
   - Sugira até **dois twists** ou obstáculos inesperados que aumentem a tensão, mas sem impedir a conclusão do objetivo atual.

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""



narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Indique o próximo local com uma ação no imperativo e e conecte diretamente ao clímax da trama principal\"\n,"
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [114]:
# Cena 4: Final da Aventura
introducao_completa = texto_introducao
local_evento_atual = "Subir as escadas para encontrar o encontro com a figura encapuzada"
# {local} e envolve {evento}

prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Cena Final: Desfecho da Aventura
Esta é a última cena da história, onde o personagem encara seu maior desafio e resolve (ou não) o problema central de forma definitiva. Não haverá novas cenas ou locais. O foco está no **clímax** e no encerramento da trama.

### Evento Final
A história chega ao seu ápice em {local_evento_atual}, onde o personagem enfrenta seu maior desafio. 
Esta é a última chance de resolver o problema principal de forma definitiva, determinando o destino da jornada e suas consequências.

### Instruções
1. **Local Atual**: Descreva o cenário com **detalhes sensoriais ricos** (visuais, sons, cheiros) para criar uma atmosfera tensa e imersiva, adequada ao confronto final.
2. **Evento Principal**: Apresente o **desafio decisivo** que o personagem enfrenta neste local. Inclua personagens envolvidos e a situação que leva ao desfecho.
3. **Objetivo do Personagem**: Descreva a **ação final** que o personagem deve realizar para resolver o conflito principal. Inicie a frase com o verbo no infinitivo.
4. **Conclusão e Consequências**: Resolva a história e explique as **consequências** das ações do personagem. Deixe claro se ele **alcança seu objetivo** ou não e quais são os impactos finais na trama.
5. **Twists Finais**: Sugira até **dois twists** ou revelações surpreendentes para tornar o final mais interessante, mas que não abram novos caminhos para a história.

**Importante**: 
- Esta é a **última cena** da aventura, e a história deve **terminar aqui**. Não deve haver conexão com um próximo local ou evento.
- O desfecho precisa resolver todos os pontos principais da trama.

### Saída
"""
narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Explique como a história termina e as consequências das ações do personagem.\"\n," # Na hora de passar isso para o modelo, isso deve ser chamado de concluisão. Não mudei o nome aui só para não precisar ceiar uma noca classe.
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [124]:
class Twist(BaseModel):
    description: str

class Personagem(BaseModel):
    name: str
    role: str
    motivation: str
    participation: str

class Item(BaseModel):
    name: str
    function: str
    condition_use: str
    status: str

class Assunto(BaseModel):
    name: str
    description: str
    status: str

class Location(BaseModel):
    name_locale: str
    scene_description: str
    objetivo_description: str
    Connection_next_location: str
    itens: List[Item]
    personagens: List[Personagem]
    assuntos: List[Assunto]
    twists: List[Twist]

In [324]:
# Cena 1
local_cena_1 = base_arcos.arcos[0].local
personagem_cena_1 = base_arcos.arcos[0].personagem
evento_cena_1 = base_arcos.arcos[0].evento
objetivo_cena_1 = base_arcos.arcos[0].objetivos
local_proxima_cena = base_arcos.arcos[1].local
objetivo_proxima_cena = base_arcos.arcos[1].objetivos

prompt_cena = f"""Com base nos dados fornecidos, descreva a cena inicial da aventura de RPG que apresente o mundo, o contexto e motive os jogadores.
## Introdução da história: 
{introducao.messages[0].content} # apenas as duas partes relevantes para a cena 1
{introducao.messages[1].content}

## Local da Cena 1: 
{local_cena_1}

## Personagem da Cena 1:
{personagem_cena_1}

## Evento da Cena 1: 
{evento_cena_1}

## Objetivo da Cena 1:
{objetivo_cena_1}

## Local da Próxima Cena: 
{local_proxima_cena}

## Objetivo da Próxima Cena: (objetivo inicial que leva a esse lugar, ao chegar la pode acontecer alqo que leve a outro objetivo)
{objetivo_proxima_cena}

## Instruções:
1. Apresentar o cenário e o contexto do local atual.
2. Introduzir o personagem relacionado ao local ou à trama.
3. Descrever o evento significativo que ocorre no local, conectando-o ao objetivo principal.
4. Mostrar o gancho narrativo, destacando a crise ou desafio que o personagem deve resolver.
5. Fazer uma ligação natural para a próxima cena, mencionando o próximo local e objetivo.
6. Sugira até dois twists ou obstáculos inesperados para aumentar a tensão e manter o interesse.

## Saída Esperada:  
"""

cena_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"title\": \"Um titulo para a cena.\",\n"
    "  \"description\": \"Descreva os principais pontos que o Mestre devera usar para contruir a narrativa dessa cena. Um resumo do que pode acontecer do incio ao fim conectando o final dessa cena com o local e objetivo da proxima cena.\",\n"
    "  \"twists\": [\n"
    "    {\"description\": \"Descrição da reviravolta\"}\n"
    "  ]\n"
    "}"
)

class Twist(BaseModel):
    description: str

class Cena(BaseModel):
    title: str
    description: str
    twists: List[Twist]


"""
Cena:
    cena atual:
        resumo do que acontece nessa cena, quem ira fazer o que e onde vai fazer e porque vai fazer.
        o que pode acontecer (se der certo ou errado) e pra onde vai e porque vai ( tiver dado certo ou errado)
    proxima cena:
        local_objetivo: pra onde vai e o porque vai.
    twists:

"""

'\nCena:\n    cena atual:\n        resumo do que acontece nessa cena, quem ira fazer o que e onde vai fazer e porque vai fazer.\n        o que pode acontecer (se der certo ou errado) e pra onde vai e porque vai ( tiver dado certo ou errado)\n    proxima cena:\n        local_objetivo: pra onde vai e o porque vai.\n    twists:\n\n'

In [322]:
system_prompt = """Você é um criador de narrativas para RPG de mesa. Sua tarefa é descrever cenas para ajudar o Mestre de Jogo a narrar a aventura.
1. **Foco**: Apresente o cenário, o evento principal e os personagens envolvidos, conectando-os ao objetivo da cena.
2. **Protagonistas**: Os jogadores são os personagens principais, e os NPCs (personagens fornecidos) estão lá para apoiar ou desafiar a história.
3. **Clareza**: A descrição deve ser direta e evocativa, sem excesso de detalhes, mas suficiente para criar uma cena imersiva.
4. **Conexão**: Faça uma transição natural para a próxima cena, destacando o próximo local e objetivo.

"""

In [ ]:
print(prompt_cena)

In [323]:
base_cena = model_response(system_prompt, prompt_cena, cena_schema, Cena)
base_cena.model_dump()
print(f"Titulo: {base_cena.title}")
print(f"{dividir_texto_em_linhas(base_cena.description, largura=100)}")
print(f"\nTwinst:")
for twinst in base_cena.twists:
    print(f"- {dividir_texto_em_linhas(twinst.description, largura=100)}")

Titulo: Cena Inicial: Luta pela Vida em Eldrador
A cidade-estado de Eldrador está sendo invadida por um exército de mortos. O rei Arin III,
desesperado para salvar sua capital, pede ajuda ao seu melhor alguém, Eira Shadowglow, uma
habilidosa caçadora maga que busca uma fórmula para desbloquear a magia das montanhas de
Kristalberg. Enquanto isso, o exército dos mortos se aproxima e invade Eldrador, matando inocentes e
destruindo tudo em seu caminho.

Twinst:
- Um dos soldados vivos do rei Arin III é visto conversando com um dos líderes do exército dos mortos,
sugerindo que pode haver uma trama mais profunda por trás da invasão.
- Eira descobre um mapa antigo escondido em sua habitação, apontando para as montanhas de Kristalberg
e indicando a localização da fórmula que ela busca.


Define a cena inicial (ve o que da pra simplificar, talvez de pra tirar os exemplos)

In [225]:
cena_inicial_cena = """Você é um chatbot narrador de aventuras de RPG de mesa, que usa o estilo de narração imersivo descritivo para uma descrição detalhada do ambiente, personagem ou evento com detalhes sensoriais (visão, som, cheiro, tato, etc.) e escreve as falas em uma nova linha com o estilo de trancrisção iniciando as falas com um hífen (—).
Crie uma introdução para a cena inicial em até duas mensagens, tendo como base a Introdução e Descrisão da cena.
Gere textos curtos, com poucas palavras.
Sempre finalize com um gancho claro, como: "O que vocês fazem?"
Informe o Status atual da narrativa, que pode ser: "Em andamento", "Finalizada".
Siga o formato das mensagens abaixo para criar a introdução da cena:
Exemplo 1
Mensagem 1: (uma curta descrição da Cena) 
A vila de Grenwald está em silêncio mortal. Telhados afundados, musgo escuro cobrindo as paredes e um cheiro de podridão pairando no ar. Você caminha firme, a armadura rangendo, enquanto observa as marcas de garras nas portas com olhos inquietos. Um vento frio sopra, trazendo uma sensação de que estão sendo observados.

Mensagem 2: (Ação ou fala do NPC)
Do beco, um velho caçador surge da sombra, o arco firme nas mãos. Ele olha diretamente para Você.
— "Não devia estar aqui."
Sua voz rouca ecoa, e ele aponta para as marcas de garras.
— "Ele passou por aqui há muito tempo... mas o mal nunca dorme."
O silêncio pesa. Você segura o cabo da espada. O que vocês fazem?

Exemplo 2: 
Mensagem 1: (uma curta descrição da Cena) 
A taverna "O Cangaceiro Vermelho" vibra com vozes embriagadas, cheiro de cerveja barata e lamparinas de óleo iluminando o ambiente. Vocês observam tudo de um canto, atento, enquanto um de vocês mexe em algumas sacolas deixadas para trás.

Mensagem 2: (Ação ou fala do NPC)
Um estrondo violento! A porta é arrombada, e homens em armaduras invadem a taverna.
— "Todo mundo no chão, agora!" — grita o líder dos invasores.
Clientes correm, cadeiras caem, e o cheiro de ferro e fumaça invade o salão. O que vocês fazem?"""

In [226]:
mensagens_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"title\": \"Título do cenário\",\n"
    "  \"messages\": [\n"
    "    {\"number\": 1, \"content\": \"Primeira mensagem\"},\n"
    "    {\"number\": 2, \"content\": \"Segunda mensagem\"}\n"
    "  ],\n"
    "  \"status\": \"Status atual da narrativa\"\n"
    "}"
)

In [227]:
class Message(BaseModel):
    number: int
    content: str

# Classe que representa o cenário completo
class Messages(BaseModel):
    title: str
    messages: List[Message]
    status: str

In [228]:
# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)


nome_modelo = lista_modelos[modelo_id]
print(f"{nome_modelo}\n")
narrativa = client.chat.completions.create(
    model=nome_modelo,
    messages=[
        {
            "role": "system",
            "content": f"{cena_inicial_cena}\n\n{mensagens_schema}"
        },
        {   "role": "user",
            "content": f"Introdução:\n{texto_introducao[0]}\n{texto_introducao[1]}\n{texto_introducao[2]}\n{texto_introducao[3]}\n\Twist para a aventura:\n{lista_twist}\n\nDescrisão da cena:\n{lista_cenas[0]}"
        }
    ],
    response_model= Messages,
    extra_body={"gpu_layers":-1, "temperature": 0.7},
    
)


if True:
    print(f"{narrativa.title}")
    print()
    for mensagem in narrativa.messages:
        print(dividir_texto_em_linhas(mensagem.content, largura=100))
        print("=")
    print(f"\n{narrativa.status}")
else:
    print(narrativa.choices[0].message.content)

llama3.2:3b-instruct-q8_0
O Início do Despertar

A vila de Início é uma pequena comunidade de casas de madeira e telhados de palha, cercada por uma
floresta densa e escura. A vila está preparada para enfrentar o desafio que se aproxima, com os
moradores armados com machados e lanças. O ar está cheio do cheiro de queimado e a sensação de
inquietude é palpável.
=
De repente, um rugido ecoa pela floresta e o solo começa a tremer. Os moradores da vila olham em
direção à floresta, os olhos wide de medo. Um homem grande e alto surge da floresta, com olhos
vermelhos brilhantes e uma pele escura como carvão. Ele é vestido com armadura negra e carrega um
bastão que emite uma luz azulada.
=

Em andamento
